In [2]:
# Parameters
datasource = "rt"
gcn_number = 33889
name = "S230529ay"
t0_utc="2023-05-29T18:15:00.746"
ra = 316.05 
dec = 15.79 
radius = 1.0
event_kind = "LVK"
healpix_url = "https://gracedb.ligo.org/api/superevents/S230529ay/files/bayestar.fits.gz,1"
force_online_expected = 0

In [18]:
# Not sending messages while doing search hides trial factors. We want to declare what we do in advance to avoid this.
# With high rate of alerts, we need a strategy to pick alerts for free text and structured messages. Will not send GCN Circulars about every event
# Structured
# Continue to distribute independent GRB detection GCN notices
# Will send GCN Counterpart Notices, new GCN notices (need to ), SCIMMA alerts, TNS.
# Send for all events < 1 minute
# Free-text (GCN Circular, ATel, AstroNotes?)
# * Including at least one NS & high-significance
# * Any likely EM counterpart reported (gamma-ray or not) in 
# * High-mass within 500 Mpc
# * If notice finds > 3 sigma counterpart for any event; indicating the trial factor and FAR (will depend on the rate)
# 

# Will announce policy in a GCN circular


In [8]:
from astropy.table import QTable

skymap = QTable.read(healpix_url)
skymap

import ligo.skymap.moc
import healpy

print(skymap.colnames)

try:
    skymap_flat = ligo.skymap.moc.rasterize(skymap, order=8)['PROBDENSITY']

    healpy.write_map('skymap.fits', skymap_flat, nest=True, overwrite=True)
    healpy.mollview(skymap_flat, nest=True)
except:
    skymap_flat = skymap['PROB']

healpix_url = 'skymap.fits'
healpy.write_map(healpix_url, skymap_flat, nest=False, overwrite=True)


setting the output map dtype to [dtype('>f8')]


['PROB', 'DISTMU', 'DISTSIGMA', 'DISTNORM']


In [19]:
import hashlib

key = hashlib.md5(("%.15lg %.15lg %.5lg %s"%(ra,dec,radius,healpix_url)).encode()).hexdigest()

name=name.replace("AUTO",
                t0_utc[2:4]+t0_utc[5:7]+t0_utc[8:10]+"."+
                (t0_utc[11:13] + t0_utc[14:16]) + t0_utc[17:].replace(".","")+"-"+key[:4])

name

'S230529ay'

In [20]:
import sys
import os

sys.path.append(os.environ['HOME']+"/work/integral-visibility")
import integralvisibility

import healpy
# sys.path.append(os.environ['HOME']+"/work/integral/followup/integral-counterpart")

import astropy.io.fits as pyfits
import astropy

import integralclient as ic

import dataanalysis as da
da.printhook.global_permissive_output=False

from astropy.coordinates import SkyCoord
from astropy import units as u

import json

import pandas as pd
draft_dir=os.environ['PWD']
analysis_dir=draft_dir

import os
os.environ['INTEGRAL_DDCACHE_ROOT']="/data/ddcache/"

import counterpart


import dataanalysis.core as da
import dataanalysis.context as ddcontext

In [21]:
loc_parameters=dict(
            ra=ra,
            dec=dec,            
            dra_p90=(-radius,radius),
            ddec_p90=(-radius,radius),
        )

print("loc_parameters", loc_parameters)

loc_parameters {'ra': 316.05, 'dec': 15.79, 'dra_p90': (-1.0, 1.0), 'ddec_p90': (-1.0, 1.0)}


In [22]:
import imp

da.reset()
imp.reload(counterpart)
imp.reload(counterpart.workflows)

counterpart.DataSource.datasource = datasource 

print(counterpart.DataSource().get_version())

if force_online_expected > 0:
    counterpart.DetectONExpected.force_online_expected = True

class OperationStatus(counterpart.OperationStatus):
    version="v2"    
    
    def main(self):
        self.ibis_on=True
        self.spi_on=True

def define_event():
    class Event(counterpart.Event):
        event_name=name
        event_origin=event_kind
        healpix_url=healpix_url.replace("\"","")
        loc_parameters = loc_parameters

        cached=True

        def setup(self):    

            self.gcn=dict(number=gcn_number)

            
            self.event_time=dict(
                utc=t0_utc,
            )
    return Event
        

ICE=define_event()().get()
ICE.promote()

ICE.loc_map

ICE.store_cache(ICE._da_locally_complete)

#counterpart.Event(use_event_kind=IceCubeEvent).promote()

ICE.describe_loc_regions()

#ICE.plot_map()


#######




#if len(sens.get_exceptions()) == 0:
#    sens.plot_fancy()




2023-05-29 20:53:26 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO NSIDE = 64
2023-05-29 20:53:26 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO ORDERING = NESTED in fits file
2023-05-29 20:53:26 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO INDXSCHM = IMPLICIT
2023-05-29 20:53:26 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO Ordering converted to RING


DataSource.v0.rt
1685386406.1576073 top 3594248/140189722232640 object requesting cache for [Event.v0.S230529ay.unknown.loc-hp-e591a4fa;NoneInTheInput]  requested by +Event.v0.S230529ay.unknown.loc-hp-e591a4fa direct
1685386406.157715 top 3594248/140189722232640 cached, proceeding to restore
1685386406.1577816 top 3594248/140189722232640 searching for cache starting from [CacheCounterparts of size 0 at /data/ddcache/]
Rev in hashe: None
[CacheCounterparts of size 0 at /data/ddcache/] cached path: /data/ddcache//byevent/S230529ay.unknown.loc-hp-e591a4fa/any/741e1753/
1685386406.1581032 top 3594248/140189722232640 [CacheCounterparts of size 0 at /data/ddcache/] found cache file: /data/ddcache//byevent/S230529ay.unknown.loc-hp-e591a4fa/any/741e1753//cache.pickle.gz
Rev in hashe: None
[CacheCounterparts of size 0 at /data/ddcache/] cached path: /data/ddcache//byevent/S230529ay.unknown.loc-hp-e591a4fa/any/741e1753/
1685386406.1582983 top 3594248/140189722232640 load_content is restoring fro

In [23]:
ICE.loc_parameters

{'ra': 250.31249999999997, 'dec': -30.69158768492234}

In [24]:
# from nb2workflow import nbadapter as nba

# nba.nbrun("integralallsky.ipynb", dict(t0_utc=t0_utc, rt=1))

In [25]:
counterpart.__file__

'/home/savchenk/integral_lvk/workflows/counterpart.py'

In [26]:
class CountLimits(counterpart.CountLimits):
    cached=False
    
    
cl = CountLimits().get()

1685386407.0707858 top 3594248/140189722232640 object requesting cache for [CountLimits.v2.2_span_300;NoneInTheInput]  requested by +CountLimits.v2.2_span_300 direct
1685386407.0718086 top 3594248/140189722232640 object requesting cache for [SourceAssumptions.v2;NoneInTheInput]  requested by +SourceAssumptions.v2 input_of +CountLimits.v2.2_span_300 output_required_by_parent +CountLimits.v2.2_span_300 direct
1685386407.0722706 top 3594248/140189722232640 object requesting cache for [SourceAssumptions.v2;NoneInTheInput]  requested by +SourceAssumptions.v2 output_required_by_parent +SourceAssumptions.v2 input_of +CountLimits.v2.2_span_300 output_required_by_parent +CountLimits.v2.2_span_300 direct
1685386407.073303 top 3594248/140189722232640 node [SourceAssumptions.v2;NoneInTheInput] main DONE!
1685386407.0742254 top 3594248/140189722232640 object storing in cache: [SourceAssumptions.v2;NoneInTheInput], cache [CacheCounterparts of size 0 at /data/ddcache/]
1685386407.0745575 top 3594248/

2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG odafunction.func.urifunc[3594248] INFO constructing <class 'odafunction.func.urifunc.URIipynbFunction'> from uri=file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb value=None provenance=None
2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG odafunction.func.urifunc[3594248] INFO parsing URI file:///home/savchenk/integral...(65)
2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG odafunction.func.urifunc[3594248] INFO parsed uri file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb as {'modifier': None, 'schema': 'file', 'path': '/home/savchenk/integral_lvk/workflows/integralallsky.ipynb', 'funcname': None, 'revision': None}
2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG nb2workflow.semantics[3594248] INFO input combined turtle: @prefix oda: <http://odahub.io/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf

1685386407.4449441 top 3594248/140189722232640 node [DetectNoData.v0;NoneInTheInput] main DONE!
1685386407.4458115 top 3594248/140189722232640 object storing in cache: [DetectNoData.v0;NoneInTheInput], cache [CacheCounterparts of size 0 at /data/ddcache/]
1685386407.446017 top 3594248/140189722232640 first non-transient cache [CacheCounterparts of size 0 at /data/ddcache/] storing [DetectNoData.v0;NoneInTheInput]
1685386407.446965 top 3594248/140189722232640 object requesting cache for [OperationStatus.v2;NoneInTheInput]  requested by +OperationStatus.v2 input_of +CountLimits.v2.2_span_300 output_required_by_parent +CountLimits.v2.2_span_300 direct
1685386407.4470346 top 3594248/140189722232640 cached, proceeding to restore
1685386407.447067 top 3594248/140189722232640 searching for cache starting from [CacheCounterparts of size 0 at /data/ddcache/]
Rev in hashe: None
[CacheCounterparts of size 0 at /data/ddcache/] cached path: /data/ddcache//byevent/S230529ay.2023-05-29T181500.746.loc

2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG nb2workflow.nbadapter[3594248] INFO parameter name=tstart_rel_mseconds value=300.0 python_type=<class 'float'>, owl_type=http://www.w3.org/2001/XMLSchema#float extra_ttl=

2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG nb2workflow.nbadapter[3594248] INFO interpreted tstart_rel_mseconds <class 'float'> 300.0 comment: 
2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG nb2workflow.semantics[3594248] INFO input combined turtle: @prefix oda: <http://odahub.io/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix unit: <http://odahub.io/ontology/unit#> .


<http://odahub.io/ontology#18cda7c8fe5211eda5d75800e3782719> a  .
2023-05-29 20:53:27 savchenk-NG-VN7-793G-79EG nb2workflow.semantics[3594248] INFO this variation could not be parsed: <http://odahub.io/ontology#18cda7c8fe5211eda5d75800e3782719> a  . due to at line 8 

2023-05-29T18:53:29.5 [terpart.py: 617[CountLimits.v2.2_sp; run_workfl: found as [URIipynbFunction][(*, RA=293.732, Dec=21.8967222, tstart_rel_mseconds=300.0, tstop_rel_seconds=300.0, t0_utc='2023-05-24T20:22:41.837', required_completeness=0.6, mode='rt', global_snr_threshold=3.0, negative_excesses=0)]:[file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb@]
2023-05-29T18:53:29.5 [terpart.py: 619[CountLimits.v2.2_sp; run_workfl: found parameters applied as [URIPythonFunction][()][prov: [('partial', ('args', ()), ('kwargs', {'t0_utc': '2023-05-29T18:15:00.746'}), ([URIipynbFunction][(*, RA=293.732, Dec=21.8967222, tstart_rel_mseconds=300.0, tstop_rel_seconds=300.0, t0_utc='2023-05-24T20:22:41.837', required_completeness=0.6, mode='rt', global_snr_threshold=3.0, negative_excesses=0)]:[file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb@], None))]]:[file:///tmp/urivalue//a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky]


...

2023-05-29T18:53:39.9 [terpart.py: 688[CountLimits.v2.2_sp;    get_ias: got ias data:
2023-05-29T18:53:39.9 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_lc_png 10 'ACS_lc.png'
2023-05-29T18:53:39.9 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_lc_png_content 101624 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdd5wU5f3A8c+UbdcLXKM3ARVU0BAQUCOCLWpiL0lUjCZ2TdRglB+Kxha7iUaNxthjNBpLKGJFwEIvitLrUa73bfP7Y9vM3l6/2907vu/Xy5fc7uzMM/35PlU
2023-05-29T18:53:39.9 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_rt_lc_png 10 'ACS_lc.png'
2023-05-29T18:53:39.9 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_rt_lc_png_content 101624 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdd5wU5f3A8c+UbdcLXKM3ARVU0BAQUCOC

2023-05-29 20:53:40 savchenk-NG-VN7-793G-79EG nb2workflow.semantics[3594248] INFO input combined turtle: @prefix oda: <http://odahub.io/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix unit: <http://odahub.io/ontology/unit#> .


<http://odahub.io/ontology#integralallsky_43bb6219> a t0_utc=Time(Time("2022-10-14T19:21:47").mjd - 8.632259375000002/24/3600, format='mjd').isot.replace(" ", "T") # hard x-ray .
2023-05-29 20:53:40 savchenk-NG-VN7-793G-79EG nb2workflow.semantics[3594248] INFO this variation could not be parsed: <http://odahub.io/ontology#integralallsky_43bb6219> a t0_utc=Time(Time("2022-10-14T19:21:47").mjd - 8.632259375000002/24/3600, format='mjd').isot.replace(" ", "T") # hard x-ray . due to at line 8 of <>:
Bad syntax (objectList expected) at ^ in:
"...b'#> .\n\n\n<http://odahub.io/ontology#integralallsky_43bb6219> a'^b' t0_utc=Time(Tim

2023-05-29T18:53:42.9 [terpart.py: 617[CountLimits.v2.2_sp; run_workfl: found as [URIipynbFunction][(*, RA=293.732, Dec=21.8967222, tstart_rel_mseconds=300.0, tstop_rel_seconds=300.0, t0_utc='2023-05-24T20:22:41.837', required_completeness=0.6, mode='rt', global_snr_threshold=3.0, negative_excesses=0)]:[file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb@]
2023-05-29T18:53:43.0 [terpart.py: 619[CountLimits.v2.2_sp; run_workfl: found parameters applied as [URIPythonFunction][()][prov: [('partial', ('args', ()), ('kwargs', {'t0_utc': '2023-05-29T18:15:00.746'}), ([URIipynbFunction][(*, RA=293.732, Dec=21.8967222, tstart_rel_mseconds=300.0, tstop_rel_seconds=300.0, t0_utc='2023-05-24T20:22:41.837', required_completeness=0.6, mode='rt', global_snr_threshold=3.0, negative_excesses=0)]:[file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb@], None))]]:[file:///tmp/urivalue//a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky]


2023-05-29 20:53:43 savchenk-NG-VN7-793G-79EG odafunction.executors[3594248] INFO loaded from cache [URIValue][value: {'output_nb': {'cells': [{'cel...(10870736)][uri: file:///tmp/urivalue//Executor_80c5a334//tmp/urivalue//a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky/a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky value: {'output_nb': {'cells': [{'cel...(10870736)]


2023-05-29T18:53:43.8 [terpart.py: 688[CountLimits.v2.2_sp;    get_ias: got ias data:
2023-05-29T18:53:43.8 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_lc_png 10 'ACS_lc.png'
2023-05-29T18:53:43.8 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_lc_png_content 101624 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdd5wU5f3A8c+UbdcLXKM3ARVU0BAQUCOCLWpiL0lUjCZ2TdRglB+Kxha7iUaNxthjNBpLKGJFwEIvitLrUa73bfP7Y9vM3l6/2907vu/Xy5fc7uzMM/35PlU
2023-05-29T18:53:43.8 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_rt_lc_png 10 'ACS_lc.png'
2023-05-29T18:53:43.8 [terpart.py: 690[CountLimits.v2.2_sp;    get_ias: acs_rt_lc_png_content 101624 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdd5wU5f3A8c+UbdcLXKM3ARVU0BAQUCOC

In [27]:
cl.input_datasource.datasource

'rt'

In [28]:
cl.get_count_limit('ACS', 1)

2023-05-29 20:53:44 savchenk-NG-VN7-793G-79EG odafunction.func.urifunc[3594248] INFO constructing <class 'odafunction.func.urifunc.URIipynbFunction'> from uri=file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb value=None provenance=None
2023-05-29 20:53:44 savchenk-NG-VN7-793G-79EG odafunction.func.urifunc[3594248] INFO parsing URI file:///home/savchenk/integral...(65)
2023-05-29 20:53:44 savchenk-NG-VN7-793G-79EG odafunction.func.urifunc[3594248] INFO parsed uri file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb as {'modifier': None, 'schema': 'file', 'path': '/home/savchenk/integral_lvk/workflows/integralallsky.ipynb', 'funcname': None, 'revision': None}
2023-05-29 20:53:44 savchenk-NG-VN7-793G-79EG nb2workflow.semantics[3594248] INFO input combined turtle: @prefix oda: <http://odahub.io/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf

found as [URIipynbFunction][(*, RA=293.732, Dec=21.8967222, tstart_rel_mseconds=300.0, tstop_rel_seconds=300.0, t0_utc='2023-05-24T20:22:41.837', required_completeness=0.6, mode='rt', global_snr_threshold=3.0, negative_excesses=0)]:[file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb@]
found parameters applied as [URIPythonFunction][()][prov: [('partial', ('args', ()), ('kwargs', {'t0_utc': '2023-05-29T18:15:00.746'}), ([URIipynbFunction][(*, RA=293.732, Dec=21.8967222, tstart_rel_mseconds=300.0, tstop_rel_seconds=300.0, t0_utc='2023-05-24T20:22:41.837', required_completeness=0.6, mode='rt', global_snr_threshold=3.0, negative_excesses=0)]:[file:///home/savchenk/integral_lvk/workflows/integralallsky.ipynb@], None))]]:[file:///tmp/urivalue//a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky]


2023-05-29 20:53:46 savchenk-NG-VN7-793G-79EG odafunction.executors[3594248] INFO loaded from cache [URIValue][value: {'output_nb': {'cells': [{'cel...(10870736)][uri: file:///tmp/urivalue//Executor_80c5a334//tmp/urivalue//a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky/a8e8d4a2//home/savchenk/integral_lvk/workflows/integralallsky value: {'output_nb': {'cells': [{'cel...(10870736)]


got ias data:
acs_lc_png 10 'ACS_lc.png'
acs_lc_png_content 101624 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdd5wU5f3A8c+UbdcLXKM3ARVU0BAQUCOCLWpiL0lUjCZ2TdRglB+Kxha7iUaNxthjNBpLKGJFwEIvitLrUa73bfP7Y9vM3l6/2907vu/Xy5fc7uzMM/35PlU
acs_rt_lc_png 10 'ACS_lc.png'
acs_rt_lc_png_content 101624 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdd5wU5f3A8c+UbdcLXKM3ARVU0BAQUCOCLWpiL0lUjCZ2TdRglB+Kxha7iUaNxthjNBpLKGJFwEIvitLrUa73bfP7Y9vM3l6/2907vu/Xy5fc7uzMM/35PlU
acs_rt_det_lc_png 14 'ACS_det_lc.png'
acs_rt_det_lc_png_content 285760 'iVBORw0KGgoAAAANSUhEUgAAAyAAAAJYCAYAAACadoJwAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAEAAElEQVR4nOzdeXxcV3n4/885996Z0Wp5

1155.7438622982056

In [29]:

class Sensitivities(counterpart.Sensitivities):
    minsolarangle=40
    datafile_restore_mode='url_in_object'
    cached=False
    
    
sens=Sensitivities().get()

#if len(sens.get_exceptions()) == 0:
#    sens.plot_fancy()


1685386427.5410604 top 3594248/140189722232640 object requesting cache for [Sensitivities.v0;NoneInTheInput]  requested by +Sensitivities.v0 direct
1685386427.556992 top 3594248/140189722232640 object requesting cache for [SensitivityMaps.v0;NoneInTheInput]  requested by +SensitivityMaps.v0 input_of +Sensitivities.v0 output_required_by_parent +Sensitivities.v0 direct
1685386427.557771 top 3594248/140189722232640 object requesting cache for [SourceAssumptions.v2;NoneInTheInput]  requested by +SourceAssumptions.v2 input_of +SensitivityMaps.v0 output_required_by_parent +SensitivityMaps.v0 input_of +Sensitivities.v0 output_required_by_parent +Sensitivities.v0 direct
1685386427.5578434 top 3594248/140189722232640 the object has not record of restore config
1685386427.5578911 top 3594248/140189722232640 ('analysis', None, 'SourceAssumptions.v2')
1685386427.5657341 top 3594248/140189722232640 object requesting cache for [CountLimits.v2.2_span_300;NoneInTheInput]  requested by +CountLimits.v2.

2023-05-29 20:53:48 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/ACS/response?lt=map2&mode=all&epeak=600&alpha=-0.5&ampl=1&model=compton&beta=-2.5&emin=75&emax=2000&emax_rate=20000


2023-05-29T18:53:48.4 [terpart.py: 873[Responses.v0;NoneIn;       main: {'model': 'compton', 'alpha': -0.5, 'epeak': 600, 'beta': -9, 'ampl': 1}


2023-05-29 20:53:49 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/ISGRI/response?lt=30&mode=all&epeak=600&alpha=-0.5&ampl=1&model=compton&beta=-2.5&emin=75&emax=2000&emax_rate=20000
2023-05-29 20:53:50 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/VETO/response?lt=100&mode=all&epeak=600&alpha=-0.5&ampl=1&model=compton&beta=-2.5&emin=75&emax=2000&emax_rate=20000
2023-05-29 20:53:50 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/ACS/response?lt=map2&mode=all&epeak=300&alpha=-1&ampl=1&model=band&beta=-2.5&emin=75&emax=2000&emax_rate=20000


2023-05-29T18:53:50.7 [terpart.py: 873[Responses.v0;NoneIn;       main: {'model': 'band', 'alpha': -1, 'epeak': 300, 'beta': -2.5, 'ampl': 1}


...

2023-05-29T18:53:54.9 [terpart.py: 873[Responses.v0;NoneIn;       main: {'model': 'powerlaw', 'alpha': -2, 'epeak': 300, 'beta': -2.5, 'ampl': 1}


2023-05-29 20:53:56 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/ISGRI/response?lt=30&mode=all&epeak=300&alpha=-2&ampl=1&model=powerlaw&beta=-2.5&emin=75&emax=2000&emax_rate=20000
2023-05-29 20:53:56 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/VETO/response?lt=100&mode=all&epeak=300&alpha=-2&ampl=1&model=powerlaw&beta=-2.5&emin=75&emax=2000&emax_rate=20000
2023-05-29 20:53:57 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/ACS/response?lt=map2&mode=all&epeak=300&alpha=-2&ampl=1&model=powerlaw&beta=-2.5&emin=75&emax=2000&emax_rate=20000


2023-05-29T18:53:57.1 [terpart.py: 873[Responses.v0;NoneIn;       main: {'model': 'powerlaw', 'alpha': -2, 'epeak': 300, 'beta': -2.5, 'ampl': 1}


2023-05-29 20:53:58 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/ISGRI/response?lt=30&mode=all&epeak=300&alpha=-2&ampl=1&model=powerlaw&beta=-2.5&emin=75&emax=2000&emax_rate=20000
2023-05-29 20:53:59 savchenk-NG-VN7-793G-79EG root[3594248] INFO http://cdcihn/response/api/v1.0/VETO/response?lt=100&mode=all&epeak=300&alpha=-2&ampl=1&model=powerlaw&beta=-2.5&emin=75&emax=2000&emax_rate=20000


1685386439.457675 top 3594248/140189722232640 node [Responses.v0;NoneInTheInput] main DONE!
1685386439.4583127 top 3594248/140189722232640 object storing in cache: [Responses.v0;NoneInTheInput], cache [CacheCounterparts of size 0 at /data/ddcache/]
1685386439.4585104 top 3594248/140189722232640 first non-transient cache [CacheCounterparts of size 0 at /data/ddcache/] storing [Responses.v0;NoneInTheInput]
1685386439.4591794 top 3594248/140189722232640 object requesting cache for [Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa;NoneInTheInput]  requested by +Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa input_of +SensitivityMaps.v0 output_required_by_parent +SensitivityMaps.v0 input_of +Sensitivities.v0 output_required_by_parent +Sensitivities.v0 direct
1685386439.4592245 top 3594248/140189722232640 the object was recovered with a different restore config: {'datafile_restore_mode': 'copy', 'datafile_target_dir': None} became {'datafile_restore_mode': 'url_in_object', 

In [30]:
import integralvisibility
sun=integralvisibility.ephem.Sun("2018-09-11")
#sun=ephem.Sun(ijd-ijd02-ephem.Date("2002/01/01"))
sun_coord=SkyCoord(sun.ra,sun.dec,unit=u.rad,representation_type="spherical")
sun_coord.separation(SkyCoord(144.58,-2.13,unit='deg'))

<Angle 25.4868563 deg>

In [31]:
if len(sens.get_exceptions()) == 0:
    bs=counterpart.BackgroundStabilityAssertion()
    cl=bs.input_countlimits().get()

    for k,v in cl.hk.items():
        for kk,vv in v.items():
            print(k,kk,vv['excvar'])

1685386459.3919058 top 3594248/140189722232640 object requesting cache for [CountLimits.v2.2_span_300;NoneInTheInput]  requested by +CountLimits.v2.2_span_300 direct
1685386459.392033 top 3594248/140189722232640 the object has not record of restore config
1685386459.3920712 top 3594248/140189722232640 ('analysis', ('list', ('analysis', None, 'SourceAssumptions.v2'), ('analysis', None, 'DataSource.v0.rt'), ('analysis', ('analysis', None, 'DataSource.v0.rt'), 'InstrumentSelection.v1'), ('analysis', ('list', ('analysis', None, 'DataSource.v0.rt'), ('analysis', ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'), 'DetectONExpected.v0'), ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa')), 'DetectNoData.v0'), ('analysis', ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'), 'OperationStatus.v2'), ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa')), 'CountLimits.v2.2_span_300')
ACS 1

In [32]:
import datetime
now_utc=datetime.datetime.strftime(datetime.datetime.now(),"%Y-%m-%dT%H:%M:%S")
now_ijd=float(ic.converttime("UTC",now_utc,"IJD"))

2023-05-29 20:54:19 savchenk-NG-VN7-793G-79EG root[3594248] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-29T20:54:19/IJD


In [33]:

ic_ijd=float(ic.converttime("UTC", ICE.event_time['utc'], "IJD"))


2023-05-29 20:54:20 savchenk-NG-VN7-793G-79EG root[3594248] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-29T18:15:00.746/IJD


In [34]:
da.byname('Sensitivities').get()._da_locally_complete

1685386460.8327458 top 3594248/140189722232640 object requesting cache for [Sensitivities.v0;NoneInTheInput]  requested by +Sensitivities.v0 direct
1685386460.8328516 top 3594248/140189722232640 the object has not record of restore config
1685386460.832885 top 3594248/140189722232640 ('analysis', ('analysis', ('list', ('analysis', None, 'SourceAssumptions.v2'), ('analysis', ('list', ('analysis', None, 'SourceAssumptions.v2'), ('analysis', None, 'DataSource.v0.rt'), ('analysis', ('analysis', None, 'DataSource.v0.rt'), 'InstrumentSelection.v1'), ('analysis', ('list', ('analysis', None, 'DataSource.v0.rt'), ('analysis', ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'), 'DetectONExpected.v0'), ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa')), 'DetectNoData.v0'), ('analysis', ('analysis', None, 'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'), 'OperationStatus.v2'), ('analysis', None, 'Event.v0.S230529ay.2023-05-29T1815

('analysis',
 ('analysis',
  ('list',
   ('analysis', None, 'SourceAssumptions.v2'),
   ('analysis',
    ('list',
     ('analysis', None, 'SourceAssumptions.v2'),
     ('analysis', None, 'DataSource.v0.rt'),
     ('analysis',
      ('analysis', None, 'DataSource.v0.rt'),
      'InstrumentSelection.v1'),
     ('analysis',
      ('list',
       ('analysis', None, 'DataSource.v0.rt'),
       ('analysis',
        ('analysis',
         None,
         'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'),
        'DetectONExpected.v0'),
       ('analysis',
        None,
        'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa')),
      'DetectNoData.v0'),
     ('analysis',
      ('analysis',
       None,
       'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'),
      'OperationStatus.v2'),
     ('analysis',
      None,
      'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa')),
    'CountLimits.v2.2_span_300'),
   ('analysis',
    ('analysis', None, 'DataSour

In [35]:
#da.reset()

class FinalComment(counterpart.DataAnalysis):
    input_ic=ICE

    read_caches=[]
    cached=True

    def main(self):
        self.comment=""
        
FinalComment().get()

1685386460.9021697 top 3594248/140189722232640 object requesting cache for [FinalComment.v0;NoneInTheInput]  requested by +FinalComment.v0 direct
1685386460.902382 top 3594248/140189722232640 cached, proceeding to restore
1685386460.9024692 top 3594248/140189722232640 searching for cache starting from [CacheCounterparts of size 0 at /data/ddcache/]
1685386460.9025648 top 3594248/140189722232640 cache [CacheCounterparts of size 0 at /data/ddcache/] not approved_read_cache, will restore_from_parent
1685386460.9026341 top 3594248/140189722232640 cache [CacheCounterparts of size 0 at /data/ddcache/] returns None
1685386460.9034703 top 3594248/140189722232640 object requesting cache for [Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa;NoneInTheInput]  requested by +Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa input_of +FinalComment.v0 output_required_by_parent +FinalComment.v0 direct
1685386460.9035504 top 3594248/140189722232640 the object was recovered with a differen

[FinalComment.v0;NoneInTheInput]

In [36]:
data={}

data['datesource'] = datasource


define_event()().get()

#for obj in [ic, counterpart.Sensitivities, counterpart.ScSystem, counterpart.OperationsReport, counterpart.CountLimits, counterpart.BackgroundStabilityAssertion, counterpart.OrientationComment, counterpart.FinalComment]:
for cls in [counterpart.Event, counterpart.FinalComment, counterpart.Sensitivities, counterpart.ScSystem, counterpart.OperationsReport, counterpart.CountLimits, counterpart.BackgroundStabilityAssertion, counterpart.OrientationComment, counterpart.FinalComment, counterpart.DetectionSummary]:
    obj = da.byname(cls().get_factory_name()).get()
        
    obj.datafile_restore_mode = 'url_in_object'
    data[obj.__class__.__name__]=obj.export_data(include_class_attributes=True)

2023-05-29 20:54:21 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO NSIDE = 64
2023-05-29 20:54:21 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO ORDERING = NESTED in fits file
2023-05-29 20:54:21 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO INDXSCHM = IMPLICIT


1685386461.0085285 top 3594248/140189722232640 object requesting cache for [Event.v0.S230529ay.unknown.loc-hp-e591a4fa;NoneInTheInput]  requested by +Event.v0.S230529ay.unknown.loc-hp-e591a4fa direct
1685386461.0089624 top 3594248/140189722232640 ('analysis', None, 'Event.v0.S230529ay.unknown.loc-hp-e591a4fa')
1685386461.0103378 top 3594248/140189722232640 object requesting cache for [Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa;NoneInTheInput]  requested by +Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa direct
1685386461.010792 top 3594248/140189722232640 object requesting cache for [Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa;NoneInTheInput]  requested by +Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa output_required_by_parent +Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa direct
2023-05-29T18:54:21.0 [terpart.py: 314[Event.v0.S230529ay.;    loc_map: map header INSTRUME L1
2023-05-29T18:54:21.0 [terpart.py: 314[Event.v0.S23052

2023-05-29 20:54:21 savchenk-NG-VN7-793G-79EG healpy[3594248] INFO Ordering converted to RING


2023-05-29T18:54:21.0 [terpart.py: 324[Event.v0.S230529ay.;    loc_map: healpix url NOT empty: peak at {'ra': 250.31249999999997, 'dec': -30.69158768492234}
2023-05-29T18:54:21.3 [terpart.py: 362[Event.v0.S230529ay.; describe_l: peak 250.13671875 -30.69158768492234
2023-05-29T18:54:21.4 [terpart.py: 366[Event.v0.S230529ay.; describe_l: 50 % containment
2023-05-29T18:54:21.4 [terpart.py: 367[Event.v0.S230529ay.; describe_l: RA 222.36 .. 44.65
2023-05-29T18:54:21.4 [terpart.py: 368[Event.v0.S230529ay.; describe_l: RA 27.77 .. 294.79
2023-05-29T18:54:21.4 [terpart.py: 369[Event.v0.S230529ay.; describe_l: Dec 36.5 .. 97.88
2023-05-29T18:54:21.4 [terpart.py: 370[Event.v0.S230529ay.; describe_l: Dec -67.19 .. 67.19
2023-05-29T18:54:21.6 [terpart.py: 366[Event.v0.S230529ay.; describe_l: 90 % containment
2023-05-29T18:54:21.6 [terpart.py: 367[Event.v0.S230529ay.; describe_l: RA 250.14 .. 109.69
2023-05-29T18:54:21.6 [terpart.py: 368[Event.v0.S230529ay.; describe_l: RA 0.0 .. 359.82
2023-05-29T

In [37]:
data['DetectionSummary'].get('txt'),data['DetectionSummary'].get('detection_decision',None)

('', False)

In [38]:
data['DetectionSummary']

{'default_log_level': '',
 'txt': '',
 'excess_list': [{'excess': {'FAP': 0.27625797700373744,
    'rate': 90343.5294117647,
    'rate_err': 230.5282034701084,
    'rate_overbkg': 1485.4069767441833,
    'rel_s_scale': 262.3093000149204,
    'snr': 4.724696427833223,
    'luminosity_min': 1.4072547657318353e+48,
    'luminosity_max': 5.3492194273381326e+48,
    'luminosity': 3.378237096534984e+48,
    'luminosity_err_loc': 3.040826216270775e+48,
    'luminosity_err': 8.30173793349588e+47,
    'luminosity_scaled_min': 14.072547657318353,
    'luminosity_scaled_max': 53.49219427338132,
    'luminosity_scaled': 33.78237096534984,
    'luminosity_scaled_err_loc': 30.40826216270775,
    'luminosity_scaled_err': 8.301737933495879,
    'FAP_sigma': 0.5939942571877542},
   'offset': 9,
   'scale': 1.7000000000000002},
  {'excess': {'FAP': 0.4981787428512599,
    'rate': 90560.0,
    'rate_err': 448.6027691002859,
    'rate_overbkg': 1701.7097958835075,
    'rel_s_scale': -12.115900008939207,
 

In [39]:
#data['integral']
revnum = ic.converttime("UTC", t0_utc, "REVNUM")


if False:
    

    data['integral']=dict(orbit_before=revnum,
                          orbit_after="%.4i"%(int(revnum)+1),
                          first_data_utc=""
                         )

    print("data",data['integral'])

    try:
        first_scw_utc = ic.converttime("SCWID", data['integral']['orbit_after']+"00010010", "UTC").split()[0]

        import time

        if False:
            while True:
                try:
                    firstacs = ic.get_hk(utc=first_scw_utc,target="ACS",span=5000)
                    break
                except counterpart.ic.ServiceException as e:
                    print(e)
                    time.sleep(1)


        print(first_scw_utc)
        first_scw_ijd = ic.converttime("UTC",first_scw_utc,"IJD")

        import isdcclient
        isc = isdcclient.ISDCClient()

        acs = isc.genlc("ACS",first_scw_utc,"5000",format='numpy')
        first_data_ijd = acs[:,0][acs[:,2]>0][0]
        first_data_utc = ic.converttime("IJD",first_data_ijd,"UTC")

        before_rev_ijd = float(ic.converttime("REVNUM",revnum,"IJD").split()[2])

        acs = isc.genlc("ACS","%.20lg"%(before_rev_ijd-5./24.),"%.20lg"%(4.5*3600),format='numpy')



        last_data_ijd = acs[:,0][acs[:,2]>0][-1]
        last_data_utc = ic.converttime("IJD",last_data_ijd,"UTC")

        print("last data",last_data_utc)

        last_data_ijd,last_data_utc


        data['integral']['last_data_utc'] = last_data_utc
        data['integral']['last_data_ijd'] = last_data_ijd

        data['integral']['first_data_utc'] = first_data_utc
        data['integral']['first_data_ijd'] = first_data_ijd
    except Exception as e:
        print("not yet",e)

2023-05-29 20:54:23 savchenk-NG-VN7-793G-79EG root[3594248] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-29T18:15:00.746/REVNUM


In [46]:
import requests

prophecy_summary = requests.get(f"https://www.astro.unige.ch/mmoda/dispatch-data/gw/integralhk/api/v1.0/rtlc/{t0_utc}/2?json&prophecy").json()['prophecy']

In [47]:
if 'expected_data_status' not in prophecy_summary[1]:
    data['integral']=dict(orbit_before=revnum,
                              orbit_after="%.4i"%(int(revnum)+1),
                              last_data_utc="UNKNOWN",
                              first_data_utc="UNKNOWN",
                             )
elif prophecy_summary[1]['expected_data_status'] == "OFFLINE":
    # or \
    # d['data']['rtstatus'] == "OFFLINE":

    data['integral']=dict(orbit_before=revnum,
                              orbit_after="%.4i"%(int(revnum)+1),
                              last_data_utc=str(prophecy_summary[1]['last_data_utc']),
                              first_data_utc=str(prophecy_summary[1]['next_data_utc']),
                             )

    try:
        data['integral']['first_data_ijd']=ic.converttime("UTC", prophecy_summary[1]['next_data_utc'], 'IJD')
        data['integral']['last_data_ijd']=ic.converttime("UTC", prophecy_summary[1]['last_data_utc'], 'IJD')
    except Exception as e:
        print("unable to get prophecy ijd!", prophecy_summary[1]['next_data_utc'], prophecy_summary[1]['last_data_utc'])
        data['integral']['first_data_ijd'] = "UNKNOWN"
        data['integral']['last_data_ijd'] = "UNKNOWN"
    

In [48]:
#da.byname("OrientationComment").get().export_data()

da.byname('ScSystem')._da_locally_complete

('analysis',
 ('analysis',
  None,
  'Event.v0.S230529ay.2023-05-29T181500.746.loc-hp-e591a4fa'),
 'ScSystem.v2')

In [49]:
ev=da.byname('Event')
final_peak = ev.peak()
print(final_peak,ev.loc_parameters)

[250.13671875, -30.69158768492234] {'ra': 250.31249999999997, 'dec': -30.69158768492234}


In [50]:
import requests
data['shift'] = dict(fullnames = requests.get("http://cdcihn/transients/dashboard/current-shift").text)

In [51]:
data['event_class'] = 'LIGO/Virgo'
data['datasource'] = datasource

In [53]:
try:
    data['gcn_repo_version'] = open("/repo-version").read().strip()
except:
    data['gcn_repo_version'] = "local"

In [54]:
from astropy import units as u
from astropy import constants as const


In [56]:
import numpy as np

if not hasattr(ev, 'map_header'):
    data['distance'] = None
else:
    distance_mean = ev.map_header.get('DISTMEAN'),ev.map_header.get('DISTSTD')

    if ev.map_header.get('DISTMEAN') is None or ev.map_header.get('DISTSTD') is None or data['Sensitivities'].get('summary') is None:
        data['distance'] = None
    else:
        data['distance']=dict(
            mean_Mpc = ev.map_header.get('DISTMEAN'),
            std_Mpc = ev.map_header.get('DISTSTD'),     
        )

        data['at_distance']=dict(
            at_mean=dict(),
        )

        for k,d in data['Sensitivities']['summary'].items():
            data['at_distance']['at_mean'][k]=dict()
            for r,rd in d.items():            
                data['at_distance']['at_mean'][k][r]=rd*float(data['distance']['mean_Mpc']*u.Mpc/u.cm)**2*4*np.pi
                print(k,r,rd,data['at_distance']['at_mean'][k][r])

data['distance']

hard p99_min 1.6649458305339694e-07 9.41504269245634e+47
hard p99_max 8.945176219690668e-07 5.058375741445155e+48
hard p90_min 1.6649458305339694e-07 9.41504269245634e+47
hard p90_max 8.945176219690668e-07 5.058375741445155e+48
hard p50_min 1.93627339857181e-07 1.0949363263052527e+48
hard p50_max 7.360110999440792e-07 4.1620428731142876e+48
hard p50_typical 3.372340491238325e-07 1.907012775804553e+48
hard prob_within_1p5best 0.26752341212631336 1.5128085852457069e+54
hard p99_min_acsresponse 1.4405837528941852e-10 8.146305595544721e+44
hard p99_max_acsresponse 7.7397566290364e-10 4.3767273238090454e+45
hard p90_min_acsresponse 1.4405837528941852e-10 8.146305595544721e+44
hard p90_max_acsresponse 7.7397566290364e-10 4.3767273238090454e+45
hard p50_min_acsresponse 1.675348199315993e-10 9.473866676029439e+44
hard p50_max_acsresponse 6.368289064330529e-10 3.6011810305771674e+45
soft p99_min 5.385585255460742e-07 3.0454753646708353e+48
soft p99_max 3.507729021608732e-06 1.9835731521320816e+

{'mean_Mpc': 217.3980356032381, 'std_Mpc': 71.40610517565747}

In [57]:
sens_map_soft_png=getattr(sens,'sens_map_soft_png','none')
sens_map_hard_png=getattr(sens,'sens_map_hard_png','none')

In [58]:
try:
    import odahubz
    data['links'] = [odahubz.publish(
        title="INTEGRAL observation of "+name,
        fn_png=sens_map_hard_png,
        provenance="gcn pipeline",
        keywords=["INTEGRAL", name, "GCN", "sensitiviy", "transient"],
    )]
except Exception as e:
    print("failed to publish", e)
    data['links'] = []

failed to publish No module named 'odahubz'


In [60]:
import ddpaper

from ddpaper.render import  render_draft
from ddpaper.render import get_latex_jinja_env


if len(sens.get_exceptions()) == 0:

    gcn_text = render_draft(get_latex_jinja_env(),
                 open("templates/gcn/upper_limit.txt").read(),
                 data
                )
else:
    print("found exceptions:",sens.get_exceptions())
    gcn_text = render_draft(get_latex_jinja_env(),
                 open("templates/gcn/inactive.txt").read(),
                 data
                )

print(gcn_text)

2023-05-29 21:01:16 savchenk-NG-VN7-793G-79EG ddpaper.render[3594248] INFO preprocessing template \BLOCK{set icevent=Event}
\BLOCK{set sens=Sensitivities.summary}

\BLOCK{ if datasource == "rt" }

    \BLOCK{set instruments="SPI-ACS"}

\BLOCK{ elif datasource == "nrt" }

    \BLOCK{set instruments="SPI-ACS and IBIS"}

\BLOCK{else}

    \BLOCK{ raise "datasources is undefined: "+datasource }

\BLOCK{ endif }


\BLOCK{ if DetectionSummary.detection_decision }
\BLOCK{ if DetectionSummary.detection_associated }
\VAR{event_class} \VAR{icevent.event_name}: 1 counterpart candidate from INTEGRAL \VAR{ instruments } prompt observation
\BLOCK{ else }
\BLOCK{ if DetectionSummary.detection_somewhat_associated }
\VAR{event_class} \VAR{icevent.event_name}: 1 weakly associated counterpart candidate in INTEGRAL \VAR{ instruments } prompt observation
\BLOCK{ else }
\VAR{event_class} \VAR{icevent.event_name}: No counterpart candidates in INTEGRAL \VAR{ instruments } prompt observation
\BLOCK{ endif }
\B


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% generated by template.py, please do not edit directly
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


    


LIGO/Virgo S230529ay: No counterpart candidates in INTEGRAL SPI-ACS prompt observation

NOWRAP 404 page not found

NOWRAP V. Savchenko, C. Ferrigno (ISDC/UniGE, Switzerland)
NOWRAP J. Rodi (IAPS-Roma, Italy)
NOWRAP A. Coleiro (APC, France)
NOWRAP S. Mereghetti (INAF IASF-Milano, Italy)

on behalf of the INTEGRAL multi-messenger collaboration:
https://www.astro.unige.ch/cdci/integral-multimessenger-collaboration


Using INTEGRAL/SPI-ACS realtime data (following [1]) we have performed a search for a prompt gamma-ray
counterpart of S230529ay (GCN 33889).

At the time of the event (2023-05-29 18:15:00 UTC,
hereafter T0), INTEGRAL was operating in nominal mode. The peak of the event
localization probability was at an angle of
122 deg with respect to the
spacecraft pointing axis. This orientation implies st

In [ ]:
import textwrap

In [ ]:
gcn_wrapped_text = ""

long_sep = "%c%c"%(10,10)

for block in gcn_text.split(long_sep):    
    if '%%%%%' in block:
        continue
    
    if 'NOWRAP' in block:
        text = block.replace("NOWRAP ","")
    else:
        text = "\n".join(textwrap.wrap(
            textwrap.dedent(block).strip(),
            70,initial_indent=''))
        
    gcn_wrapped_text += text+long_sep

print(gcn_wrapped_text,"\n")

In [ ]:
open("gcn.txt","w").write(gcn_text)
open("gcn_wrapped.txt","w").write(gcn_wrapped_text)

In [ ]:
# fov info
# ibas triggers
# title to what is used
# say what is used
# list counterparts
# luminosity
# strss all-sky

In [ ]:
gcn_html = gcn_wrapped_text.replace("\n","<br>")

In [ ]:
#! < gcn.txt awk '{if (/^NOWRAP/) {gsub("^NOWRAP ",""); print} else {print "WRAP "$$0}}' | fmt --prefix WRAP | awk '{gsub("^WRAP ?","")}1' > gcn_wrapped.txt
#fmt -w 70  gcn.txt > gcn_wrapped.txt

In [ ]:
try:
    data['CountLimits']['rtdata']['acs_rt_lc_png_content']='contracted'
    data['CountLimits']['rtdata']['acs_rt_det_lc_png_content']='contracted'
except Exception as e:
    print(e)

import json
    
try:
    json.dump(data, open("gcndata.json","w"))
except Exception as e:
    json.dump({}, open("gcndata.json","w"))

In [ ]:
gcn_wrapped_txt="gcn_wrapped.txt"
gcn_txt="gcn.txt"
gcn_wrapped_text=gcn_wrapped_text
gcn_text=gcn_text
gcn_html=gcn_html
sens_map_soft_png=sens_map_soft_png
sens_map_hard_png=sens_map_hard_png
gcndata_txt="gcndata.json"
final_peak=final_peak